<a href="https://colab.research.google.com/github/IFES-MPCA/redes_neurais_artificiais/blob/main/keras_classification_multiclass_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definir semente p/ funções pseudo aleatórias

In [1]:
import numpy
import tensorflow as tf
import random as python_random

SEED = 42
numpy.random.seed(SEED)
python_random.seed(SEED)
tf.random.set_seed(SEED)

# Carregamento dos dados

## Keras Datasets

In [2]:
from tensorflow.keras.datasets import mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

## Yan Lecun - MNist

Foi utilizado um [repositório](https://github.com/hsjeong5/MNIST-for-Numpy/tree/master) que encapsula o download dos datasets a partir do site do Yann Lecun, como pode ser visto no arquivo [mnist.py](https://github.com/hsjeong5/MNIST-for-Numpy/blob/master/mnist.py#L6).

In [3]:
!git clone 'https://github.com/hsjeong5/MNIST-for-Numpy'

fatal: destination path 'MNIST-for-Numpy' already exists and is not an empty directory.


In [4]:
import os 
os.chdir('./MNIST-for-Numpy/')

In [5]:
!mv mnist.py mnist_custom.py

mv: cannot stat 'mnist.py': No such file or directory


In [6]:
from mnist_custom import load, init

init()
custom_mnist_train_x, custom_mnist_train_y, custom_mnist_test_x, custom_mnist_test_y = load()

Download complete.
Save complete.


# Rede customizada

## Definição do modelo

In [7]:
from keras.utils.sidecar_evaluator import optimizer
from prompt_toolkit.shortcuts.progress_bar.base import E
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow import keras
from tensorflow.keras import layers 

class NeuralNetwork(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128):
    self.epochs = epochs
    self.batch_size = batch_size

  def build_model(self, y_hot_encoded):
    return keras.Sequential([
        layers.Conv2D(4, 4, activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(y_hot_encoded.shape[1], activation="softmax")
    ])
    
  def fit(self, x, y):
    self.labels, ids = numpy.unique(y, return_inverse=True)
    y_hot_encoded = keras.utils.to_categorical(ids)
    self.model = self.build_model(y_hot_encoded)
    self.model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    self.model.fit(x, y_hot_encoded, epochs=self.epochs, batch_size=self.batch_size)
    return self 
  
  def predict(self, x=None):
    predictions = self.model.predict(x)
    return self.labels[numpy.argmax(predictions, axis=1)]

  def evaluate(self, x=None, y=None):
    return self.model.evaluate(x, y)

## Utilitários para pipeline

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class NormalizeImage(BaseEstimator, TransformerMixin):
  def fit(self, X, y):
    return self
  
  def transform(self, X, y=None):
    """Normaliza a imagem de entrada (RGB) para valores entre 0 e 1"""
    return X.astype("float32") / 255

  def evaluate(self, x=None, y=None):
    return self

class ResizeMNist(BaseEstimator, TransformerMixin):
  size = 28

  def fit(self, X, y):
    return self
  
  def transform(self, X, y=None):
    return X.reshape((-1, self.size, self.size, 1))
  
  def evaluate(self, x=None, y=None):
    return self

# Treinamento

## Dataset Tensorflow Keras

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

model_keras_ds = Pipeline([
    ("scaler", NormalizeImage()), 
    ("reshape", ResizeMNist()), 
    ("ann", NeuralNetwork())
])
model_keras_ds.fit(train_x, train_y)

Epoch 1/5
469/469 [==============================] - 37s 75ms/step - loss: 0.2170 - accuracy: 0.9349
Epoch 2/5
469/469 [==============================] - 29s 62ms/step - loss: 0.0717 - accuracy: 0.9782
Epoch 3/5
469/469 [==============================] - 35s 75ms/step - loss: 0.0444 - accuracy: 0.9864
Epoch 4/5
469/469 [==============================] - 30s 64ms/step - loss: 0.0294 - accuracy: 0.9909
Epoch 5/5
469/469 [==============================] - 29s 62ms/step - loss: 0.0191 - accuracy: 0.9941


Pipeline(steps=[('scaler', NormalizeImage()), ('reshape', ResizeMNist()),
                ('ann', NeuralNetwork())])

## Dataset Yann Lecun

In [10]:
model_lecun_ds = Pipeline([
    ("scaler", NormalizeImage()), 
    ("reshape", ResizeMNist()), 
    ("ann", NeuralNetwork())
])
model_lecun_ds.fit(custom_mnist_train_x, custom_mnist_train_y)

Epoch 1/5
469/469 [==============================] - 31s 64ms/step - loss: 0.2127 - accuracy: 0.9352
Epoch 2/5
469/469 [==============================] - 31s 67ms/step - loss: 0.0694 - accuracy: 0.9787
Epoch 3/5
469/469 [==============================] - 29s 63ms/step - loss: 0.0416 - accuracy: 0.9866
Epoch 4/5
469/469 [==============================] - 29s 62ms/step - loss: 0.0270 - accuracy: 0.9918
Epoch 5/5
469/469 [==============================] - 29s 62ms/step - loss: 0.0177 - accuracy: 0.9944


Pipeline(steps=[('scaler', NormalizeImage()), ('reshape', ResizeMNist()),
                ('ann', NeuralNetwork())])

# Comparação

## Dataset Yann Lecun

In [11]:
predictions = model_lecun_ds.predict(custom_mnist_test_x)
accuracy_score(custom_mnist_test_y, predictions)

313/313 [==============================] - 2s 7ms/step


0.9863

## Dataset Tensorflow Keras

In [12]:
predictions = model_keras_ds.predict(test_x)
accuracy_score(test_y, predictions)

313/313 [==============================] - 2s 7ms/step


0.9828